Data Processing

Objective:
Clean and prepare the Titanic Dataset for feature engineering and modeling

Tasks:
1. Load raw data
2. Handle missing values
3. Encode categorical variables
4. Validate data quality 
5. Save cleaned dataset for next phase

Setup and Imports

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

warnings.filterwarnings('ignore')

Set display options

In [2]:
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')

Load Raw datasets
We start fresh from the raw data. Kepps the notebook self-contained and reproducible. 

In [3]:
train_df = pd.read_csv('../data/raw/train.csv')
test_df = pd.read_csv('../data/raw/test.csv')

# Store PassengerId for later 
test_passenger_ids = test_df['PassengerId'].copy()

print(f"Training set: {train_df.shape}")
print(f"Test set: {test_df.shape}")

Training set: (891, 12)
Test set: (418, 11)


Check for missing values before we start

In [5]:
print("=" * 50)
print("MISSING VALUES - TRAINING SET")
print("=" * 50)
missing_train = train_df.isnull().sum()
missing_pct = (missing_train / len(train_df) * 100).round(2)
missing_summary = pd.DataFrame({
    'Missing Count': missing_train,
    'Missing %': missing_pct
})
print(missing_summary[missing_summary['Missing Count'] > 0])

MISSING VALUES - TRAINING SET
          Missing Count  Missing %
Age                 177      19.87
Cabin               687      77.10
Embarked              2       0.22


Handling Missing Values

Based on EDA findings:
Age: ~20% missing -> Impute using median by Pclass and Sex
Cabin: ~ 77% missing -> Extract deck letter, fill rest with 'Unknown'
Embarked: 2 missing values -> Fill with mode

Embarked

In [6]:
print("Rows with missing Embarked:")
print(train_df[train_df['Embarked'].isnull()])

Rows with missing Embarked:
     PassengerId  Survived  Pclass                                       Name  \
61            62         1       1                        Icard, Miss. Amelie   
829          830         1       1  Stone, Mrs. George Nelson (Martha Evelyn)   

        Sex   Age  SibSp  Parch  Ticket  Fare Cabin Embarked  
61   female  38.0      0      0  113572  80.0   B28      NaN  
829  female  62.0      0      0  113572  80.0   B28      NaN  


In [ ]:
#Find the mode for Embarked
embarked_mode = train_df['Embarked'].mode()[0]
print(f"Most common embarkation port: {embarked_mode}")
print(f"\nEmbarked distribution:")
print(train_df['Embarked'].value_counts())

Most common embarkation port: S

Embarked distribution:
Embarked
S    644
C    168
Q     77
Name: count, dtype: int64


In [9]:
# Fill missing Embarked with mode
train_df['Embarked'] = train_df['Embarked'].fillna(embarked_mode)
test_df['Embarked'] = test_df['Embarked'].fillna(embarked_mode)

print(f"Embarked missing values after imputation: {train_df['Embarked'].isnull().sum()}")

Embarked missing values after imputation: 0


Age

In [10]:
# Calculate median age by Pclass and Sex
age_medians = train_df.groupby(['Pclass', 'Sex'])['Age'].median()
print("Median ages by Pclass and Sex:")
print(age_medians)

Median ages by Pclass and Sex:
Pclass  Sex   
1       female    35.0
        male      40.0
2       female    28.0
        male      30.0
3       female    21.5
        male      25.0
Name: Age, dtype: float64


In [11]:
def impute_age(df, age_medians):
    """
    Impute missing ages using median by Pclass and Sex.
    Returns a copy with imputed values.

    Args:
        df (_type_): _description_
        age_medians (_type_): _description_
    """
    df = df.copy()
    
    for (pclass, sex), median_age in age_medians.items():
        mask = (df['Age'].isnull()) & (df['Pclass'] == pclass) & (df['Sex'] == sex)
        df.loc[mask, 'Age'] = median_age
    
    
    overall_median = df['Age'].median()
    df['Age'] = df['Age'].fillna(overall_median)
    
    return df


train_df = impute_age(train_df, age_medians)
test_df = impute_age(test_df, age_medians)

print(f"Age missing values after imputation:")
print(f"  Train: {train_df['Age'].isnull().sum()}")
print(f"  Test: {test_df['Age'].isnull().sum()}")

Age missing values after imputation:
  Train: 0
  Test: 0


In [15]:
# Visualization age distribution before and after
original_train = pd.read_csv('../data/raw/train.csv')

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(original_train['Age'].dropna(), bins=30, edgecolor='black', alpha=0.7)
axes[0].set_title('Age Distribution (Before Imputation)')
axes[0].set_xlabel('Age')
axes[0].set_ylabel('Count')

axes[1].hist(train_df['Age'], bins=30, edgecolor='black', alpha=0.7, color='green')
axes[1].set_title('Age Distribution (After Imputation)')
axes[1].set_xlabel('Age')
axes[1].set_ylabel('Count')

plt.tight_layout()
plt.savefig('../reports/figures/age_imputation_comparison.png')
plt.close()

del original_train

Cabin - Extract Deck Information

While 77% of Cabin data is missing, we can still extract useful information:
The deck letter (A, B, C, etc.) indicates location on ship
Having cabin info at all may indicate higher status passengers

In [17]:
def extract_deck(cabin):
    """
    Extract deck letter from cabin number.
    Returns 'Unknown' if cabin is missing.

    Args:
        cabin (_type_): _description_
    """
    
    if pd.isnull(cabin):
        return 'Unknown'
    return cabin[0]


train_df['Deck'] = train_df['Cabin'].apply(extract_deck)
test_df['Deck'] = test_df['Cabin'].apply(extract_deck)

print("Deck distribution (Training):")
print(train_df['Deck'].value_counts())

Deck distribution (Training):
Deck
Unknown    687
C           59
B           47
D           33
E           32
A           15
F           13
G            4
T            1
Name: count, dtype: int64


In [18]:
# Check survival rate by deck
deck_survival = train_df.groupby('Deck')['Survived'].agg(['mean', 'count'])
deck_survival.columns = ['Survival Rate', 'Count']
deck_survival = deck_survival.sort_values('Survival Rate', ascending=False)
print("\nSurvival rate by deck:")
print(deck_survival)


Survival rate by deck:
         Survival Rate  Count
Deck                         
D             0.757576     33
E             0.750000     32
B             0.744681     47
F             0.615385     13
C             0.593220     59
G             0.500000      4
A             0.466667     15
Unknown       0.299854    687
T             0.000000      1


Fare - Check for missing values in test set

In [19]:
print(f"Missing Fare in test set: {test_df['Fare'].isnull().sum()}")

if test_df['Fare'].isnull().sum() > 0:
    # Impute with median fare for that Pclass
    for pclass in test_df['Pclass'].unique():
        median_fare = train_df[train_df['Pclass'] == pclass]['Fare'].median()
        mask = (test_df['Fare'].isnull()) & (test_df['Pclass'] == pclass)
        test_df.loc[mask, 'Fare'] = median_fare
    print(f"Missing Fare after imputation: {test_df['Fare'].isnull().sum()}")

Missing Fare in test set: 1
Missing Fare after imputation: 0


Encode Categorical Variables 

In [20]:
# Sex : Binary encoding (female = 1, male = 0)
#Makes interpretation easier: positive coefficient = higher survival for females

train_df['Sex_encoded'] = train_df['Sex'].map({'female': 1, 'male': 0})
test_df['Sex_encoded'] = test_df['Sex'].map({'female': 1, 'male': 0})

print("Sex encoding:")
print(train_df[['Sex', 'Sex_encoded']].drop_duplicates())

Sex encoding:
      Sex  Sex_encoded
0    male            0
1  female            1


In [21]:
# Embarked: One-hot encoding
embarked_dummies_train = pd.get_dummies(train_df['Embarked'], prefix='Embarked')
embarked_dummies_test = pd.get_dummies(test_df['Embarked'], prefix='Embarked')

train_df = pd.concat([train_df, embarked_dummies_train], axis=1)
test_df = pd.concat([test_df, embarked_dummies_test], axis=1)

print("Embarked one-hot columns added:")
print([col for col in train_df.columns if 'Embarked_' in col])

Embarked one-hot columns added:
['Embarked_C', 'Embarked_Q', 'Embarked_S']


In [22]:
# Deck: One-hot encoding
deck_dummies_train = pd.get_dummies(train_df['Deck'], prefix='Deck')
deck_dummies_test = pd.get_dummies(test_df['Deck'], prefix='Deck')

# Ensure both have same columns (test might be missing some decks)
for col in deck_dummies_train.columns:
    if col not in deck_dummies_test.columns:
        deck_dummies_test[col] = 0

# Reorder test columns to match train
deck_dummies_test = deck_dummies_test[deck_dummies_train.columns]

train_df = pd.concat([train_df, deck_dummies_train], axis=1)
test_df = pd.concat([test_df, deck_dummies_test], axis=1)

print(f"Deck columns added: {len(deck_dummies_train.columns)}")

Deck columns added: 9


Drop Redunant and Unused Columns

In [27]:
# Columns to drop after encoding
columns_to_drop = [
    'Sex',        # Replaced by Sex_encoded
    'Embarked',   # Replaced by Embarked_* one-hot columns
    'Cabin',      # Replaced by Deck_* one-hot columns
    'Deck',       # Replaced by Deck_* one-hot columns
    'PassengerId',# Just an identifier
    'Ticket',     # Messy, not using for now
]


train_df = train_df.drop(columns=columns_to_drop)
test_df = test_df.drop(columns=columns_to_drop)

print(f"Dropped columns: {columns_to_drop}")
print(f"\nRemaining columns: {list(train_df.columns)}")

Dropped columns: ['Sex', 'Embarked', 'Cabin', 'Deck', 'PassengerId', 'Ticket']

Remaining columns: ['Survived', 'Pclass', 'Name', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_encoded', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Deck_A', 'Deck_B', 'Deck_C', 'Deck_D', 'Deck_E', 'Deck_F', 'Deck_G', 'Deck_T', 'Deck_Unknown']


Data Quality Validation

In [28]:
print("=" * 50)
print("FINAL MISSING VALUE CHECK")
print("=" * 50)

print("\nTraining set:")
train_missing = train_df.isnull().sum()
if train_missing.sum() > 0:
    print(train_missing[train_missing > 0])
else:
    print("No missing values!")

print("\nTest set:")
test_missing = test_df.isnull().sum()
if test_missing.sum() > 0:
    print(test_missing[test_missing > 0])
else:
    print("No missing values!")

FINAL MISSING VALUE CHECK

Training set:
No missing values!

Test set:
No missing values!


In [29]:
# Check data types
print("\nData types (Training):")
print(train_df.dtypes)


Data types (Training):
Survived          int64
Pclass            int64
Name             object
Age             float64
SibSp             int64
Parch             int64
Fare            float64
Sex_encoded       int64
Embarked_C         bool
Embarked_Q         bool
Embarked_S         bool
Deck_A             bool
Deck_B             bool
Deck_C             bool
Deck_D             bool
Deck_E             bool
Deck_F             bool
Deck_G             bool
Deck_T             bool
Deck_Unknown       bool
dtype: object


In [30]:
# Preview the cleaned data
print("\nCleaned dataset preview:")
print(train_df.head())


Cleaned dataset preview:
   Survived  Pclass                                               Name   Age  \
0         0       3                            Braund, Mr. Owen Harris  22.0   
1         1       1  Cumings, Mrs. John Bradley (Florence Briggs Th...  38.0   
2         1       3                             Heikkinen, Miss. Laina  26.0   
3         1       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)  35.0   
4         0       3                           Allen, Mr. William Henry  35.0   

   SibSp  Parch     Fare  Sex_encoded  Embarked_C  Embarked_Q  Embarked_S  \
0      1      0   7.2500            0       False       False        True   
1      1      0  71.2833            1        True       False       False   
2      0      0   7.9250            1       False       False        True   
3      1      0  53.1000            1       False       False        True   
4      0      0   8.0500            0       False       False        True   

   Deck_A  Deck_B  Deck_C  Dec

In [31]:
# List all columns now available
print(f"\nTotal columns: {len(train_df.columns)}")
print("\nColumn list:")
for i, col in enumerate(train_df.columns, 1):
    print(f"  {i:2d}. {col}")


Total columns: 20

Column list:
   1. Survived
   2. Pclass
   3. Name
   4. Age
   5. SibSp
   6. Parch
   7. Fare
   8. Sex_encoded
   9. Embarked_C
  10. Embarked_Q
  11. Embarked_S
  12. Deck_A
  13. Deck_B
  14. Deck_C
  15. Deck_D
  16. Deck_E
  17. Deck_F
  18. Deck_G
  19. Deck_T
  20. Deck_Unknown


Save Processed datasets

In [32]:
# Save preprocessed datasets
train_df.to_csv('../data/processed/train_preprocessed.csv', index=False)
test_df.to_csv('../data/processed/test_preprocessed.csv', index=False)

# Also save the test passenger IDs separately (needed for submission)
test_passenger_ids.to_csv('../data/processed/test_passenger_ids.csv', index=False)

print("Saved preprocessed data:")
print("  - ../data/processed/train_preprocessed.csv")
print("  - ../data/processed/test_preprocessed.csv")
print("  - ../data/processed/test_passenger_ids.csv")

Saved preprocessed data:
  - ../data/processed/train_preprocessed.csv
  - ../data/processed/test_preprocessed.csv
  - ../data/processed/test_passenger_ids.csv


Summary

What we accomplished:
1. **Embarked**: Filled 2 missing values with mode ('S' - Southampton)
2. **Age**: Imputed ~20% missing values using median by Pclass and Sex
3. **Cabin**: Extracted Deck information, labeled missing as 'Unknown'
4. **Fare**: Checked and imputed any missing values in test set
5. **Encoded**: Sex (binary), Embarked (one-hot), Deck (one-hot)

Next steps (03_feature_engineering.ipynb):
Extract titles from names (Mr, Mrs, Miss, etc.)
Create family size feature (SibSp + Parch + 1)
Create 'IsAlone' binary feature
Bin Age and Fare into categories
Create interaction features